In [ ]:
import os
os.chdir("/home/yyb02274/yolov5")  # 네 yolov5 루트 경로
print("CWD:", os.getcwd())

from models.yolo import Model


In [ ]:
from models.yolo import Model
import yaml
import torch

# 설정
cfg_path = 'models/yolov5s.yaml'
weights_path = 'runs/train/baseline_yolov5s/weights/best.pt'
num_classes = 80
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 구조 생성
with open(cfg_path) as f:
    model_cfg = yaml.safe_load(f)

model = Model(model_cfg, ch=3, nc=num_classes).to(device)
model.eval()

checkpoint = torch.load(weights_path, map_location=device, weights_only=False)
state_dict = checkpoint['model'].state_dict()
model.load_state_dict(state_dict)
print("✅ 모델 구조 및 가중치 로딩 완료")


In [ ]:
import torch.nn.utils.prune as prune
import torch.nn as nn

prune_rate = 0.5
modules_to_prune = []

print(f"L1 Unstructured Pruning 시작 (비율: {prune_rate * 100:.0f}%)")

for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        print(f" → {name} pruning 적용 중...")
        prune.l1_unstructured(module, name='weight', amount=prune_rate)
        modules_to_prune.append((module, 'weight'))

print("가지치기 임시 적용 완료 (reparametrization 상태)")


In [ ]:
for module, param_name in modules_to_prune:
    
    if hasattr(module, param_name + "_mask"):
        prune.remove(module, param_name)

print("✅ pruning 결과를 모델에 영구 반영했습니다 (weight_mask 제거 완료)")

print("weight_orig 존재 여부 확인 (상위 3개):")
for module, _ in modules_to_prune[:3]:
    print('weight_orig' in module._parameters)


In [ ]:
import torch

ckpt_path = "runs/train/baseline_yolov5s/weights/pruned_ratio[0.5].pt"
torch.save({'model': model}, ckpt_path)
print(f"📦 YOLOv5용 checkpoint 저장 완료: {ckpt_path}")


In [ ]:
def calculate_global_sparsity(model):
    total_zeros = 0
    total_elements = 0
    for name, param in model.named_parameters():
        if 'weight' in name and param.requires_grad:
            total_zeros += torch.sum(param == 0).item()
            total_elements += param.numel()
    sparsity = 100.0 * total_zeros / total_elements
    return sparsity, total_zeros, total_elements

sparsity, total_zeros, total_elements = calculate_global_sparsity(model)
print(f"📉 전체 sparsity: {sparsity:.2f}%")
print(f"   → 0인 weight 수: {total_zeros:,} / 전체 weight 수: {total_elements:,}")


In [ ]:
import numpy as np

def print_nonzeros(model):
    nonzero = total = 0
    for name, p in model.named_parameters():
        if 'mask' in name:
            continue  # pruning mask는 분석 대상에서 제외
        tensor = p.data.cpu().numpy()
        nz_count = np.count_nonzero(tensor)
        total_params = np.prod(tensor.shape)
        nonzero += nz_count
        total += total_params
        print(f'{name:40} | nonzeros = {nz_count:9} / {total_params:9} '
              f'({100 * nz_count / total_params:6.2f}%) | '
              f'total_pruned = {total_params - nz_count:9} | shape = {tensor.shape}')
    
    pruned = total - nonzero
    compression = total / nonzero if nonzero > 0 else float('inf')
    pruned_percent = 100 * pruned / total
    print("-" * 100)
    print(f'TOTAL → alive: {nonzero:,}, pruned: {pruned:,}, total: {total:,}')
    print(f'       Compression rate: {compression:10.2f}x   ({pruned_percent:6.2f}% pruned)')

print_nonzeros(model)


In [ ]:
with open("hyp_custom.yaml", "w") as f:
    f.write("""lr0: 0.001
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.20
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0
""")


In [ ]:
!python train.py \
  --weights runs/train/baseline_yolov5s/weights/pruned_ratio[0.5].pt\
  --data data/coco128.yaml \
  --cfg models/yolov5s.yaml \
  --hyp hyp_custom.yaml \
  --epochs 20 \
  --batch-size 16 \
  --name retrain_pruned \
  --exist-ok
  #까지는 정상작동!

In [ ]:
def freeze_mask_to_weight(model):
    num_applied = 0
    for m in model.modules():
        if hasattr(m, "weight") and hasattr(m, "mask"):
            with torch.no_grad():
                m.weight.data.mul_(m.mask)
            num_applied += 1
    print(f"[freeze_mask_to_weight] applied to {num_applied} layers.")

# pruned_ckpt 불러오기
import torch
ckpt = torch.load("runs/train/baseline_yolov5s/weights/pruned_ratio[0.5].pt", map_location="cpu", weights_only=False)
model = ckpt['model'] if 'model' in ckpt else ckpt  # YOLOv5 구조일 경우 그대로 사용

freeze_mask_to_weight(model)


In [ ]:
import numpy as np, json, torch
from pathlib import Path

# 저장 위치
SAVE_DIR = Path("csr_dump_ratio[0.5]")
SAVE_DIR.mkdir(exist_ok=True)

# 0이 아닌 값으로 간주할 임계값 (완전히 0이면 0.0, 혹은 1e-12)
THRESH = 0.0

def to_csr_2d(mat: np.ndarray, thresh=0.0):
    """2D numpy array -> CSR(row_ptr, col_ind, values)"""
    rows, cols = mat.shape
    row_ptr = [0]
    col_ind, values = [], []
    nnz = 0
    for r in range(rows):
        nz_cols = np.nonzero(np.abs(mat[r]) > thresh)[0]
        col_ind.extend(nz_cols.tolist())
        values.extend(mat[r, nz_cols].tolist())
        nnz += len(nz_cols)
        row_ptr.append(nnz)
    return np.array(row_ptr, np.int64), np.array(col_ind, np.int32), np.array(values, np.float32)

def export_model_to_csr(model, save_dir, thresh=0.0):
    """Conv/Linear 레이어를 CSR로 저장하고 manifest.json 생성"""
    manifest = {"layers": []}

    for name, m in model.named_modules():
        if not (hasattr(m, "weight") and isinstance(getattr(m, "weight", None), torch.Tensor)):
            continue

        W = m.weight.detach().cpu().numpy()

        # Conv
        if W.ndim == 4:
            O, I, kH, kW = W.shape
            W2 = W.reshape(O, I * kH * kW)
            shape2d, orig_shape, kind = (O, I*kH*kW), (O, I, kH, kW), "conv2d"
        # Linear
        elif W.ndim == 2:
            W2 = W
            shape2d, orig_shape, kind = W.shape, W.shape, "linear"
        else:
            continue

        # CSR 변환
        rp, ci, val = to_csr_2d(W2, thresh)

        # 안전한 파일 이름
        safe_name = name.replace(".", "_")
        np.savez(save_dir / f"{safe_name}.npz",
                 row_ptr=rp, col_ind=ci, values=val)

        # 메타데이터 기록
        manifest["layers"].append({
            "name": name,
            "type": kind,
            "shape": list(orig_shape),
            "shape2d": list(shape2d),
            "nnz": int(val.size),
            "density": float(val.size) / float(np.prod(shape2d))
        })

    # manifest 저장
    with open(save_dir / "manifest_ratio[0.5].json", "w") as f:
        json.dump(manifest, f, indent=2)

    print(f"✅ Saved CSR dump to: {save_dir.resolve()} | layers: {len(manifest['layers'])}")

# 실행
export_model_to_csr(model, SAVE_DIR, thresh=THRESH)


In [ ]:
# === Round-trip 검증: CSR -> dense 복원 == 원본 weight ? ===
import json, numpy as np
from pathlib import Path

SAVE_DIR = Path("csr_dump_ratio[0.5]")     # CSR 덤프 위치
THRESH = 0.0                    # 내보낼 때 썼던 임계값과 동일하게
MAX_PRINT = 10                  # 미스매치 최대 표시 개수

def csr_to_dense(row_ptr, col_ind, values, shape2d):
    rows, cols = shape2d
    dense = np.zeros((rows, cols), dtype=np.float32)
    for r in range(rows):
        s, e = int(row_ptr[r]), int(row_ptr[r+1])
        cols_r = col_ind[s:e]
        vals_r = values[s:e]
        dense[r, cols_r] = vals_r
    return dense

# manifest 불러오기
mani_path = SAVE_DIR / "manifest_ratio[0.5].json"
assert mani_path.exists(), f"manifest_ratio[0.5].json not found in {SAVE_DIR}"
manifest = json.load(open(mani_path, "r", encoding="utf-8"))

# model의 모듈 dict (이름→모듈)
module_map = dict(model.named_modules())

checked, mismatches = 0, []

for meta in manifest["layers"]:
    name   = meta["name"]
    shape2 = tuple(meta["shape2d"])
    kind   = meta["type"]

    # CSR 파일 로드 (export 시 '.' -> '_'로 저장했음)
    npz_path = SAVE_DIR / f"{name.replace('.', '_')}.npz"
    z = np.load(npz_path)
    dense = csr_to_dense(z["row_ptr"], z["col_ind"], z["values"], shape2)

    # 원본 가중치 꺼내서 2D로 펴기
    mod = module_map.get(name, None)
    if mod is None or not hasattr(mod, "weight"):
        mismatches.append((name, "module-not-found"))
        continue

    W = mod.weight.detach().cpu().numpy()
    if kind == "conv2d":
        O, I, kH, kW = meta["shape"]
        W2 = W.reshape(O, I * kH * kW)
    else:
        W2 = W  # linear

    # 비교: (1) nnz 동일? (2) 값 완전 동일?
    nnz_W2 = int(np.count_nonzero(np.abs(W2) > THRESH))
    nnz_csr = int(z["values"].size)

    same_nnz = (nnz_W2 == nnz_csr)
    same_vals = np.allclose(W2, dense, atol=0.0, rtol=0.0)

    if not (same_nnz and same_vals):
        reason = []
        if not same_nnz: reason.append(f"nnz {nnz_W2} != {nnz_csr}")
        if not same_vals: reason.append("values differ")
        mismatches.append((name, ", ".join(reason)))
    checked += 1

# 결과 출력
print(f"Checked layers: {checked}")
if mismatches:
    print(f"❌ Mismatches: {len(mismatches)} (showing up to {MAX_PRINT})")
    for n, r in mismatches[:MAX_PRINT]:
        print(f" - {n}: {r}")
else:
    print("✅ All matched perfectly (CSR round-trip OK)")
